In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import h5py
subhalo = h5py.File('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127/fof_subhalo_tab_127.0.hdf5','r')

def printname(name):
    print (name)
    
subhalo.visit(printname)

Config
Group
Group/GroupBHMass
Group/GroupBHMdot
Group/GroupCM
Group/GroupFirstSub
Group/GroupGasMetalFractions
Group/GroupGasMetallicity
Group/GroupLen
Group/GroupLenType
Group/GroupMass
Group/GroupMassType
Group/GroupNsubs
Group/GroupPos
Group/GroupRadioLuminosity
Group/GroupSFR
Group/GroupStarMetalFractions
Group/GroupStarMetallicity
Group/GroupVel
Group/GroupWindMass
Group/GroupXrayLuminosity
Group/Group_M_Crit200
Group/Group_M_Crit500
Group/Group_M_Mean200
Group/Group_M_TopHat200
Group/Group_R_Crit200
Group/Group_R_Crit500
Group/Group_R_Mean200
Group/Group_R_TopHat200
Header
IDs
Parameters
Subhalo
Subhalo/SubhaloBHMass
Subhalo/SubhaloBHMdot
Subhalo/SubhaloBfldDisk
Subhalo/SubhaloBfldHalo
Subhalo/SubhaloCM
Subhalo/SubhaloGasMetalFractions
Subhalo/SubhaloGasMetalFractionsHalfRad
Subhalo/SubhaloGasMetalFractionsMaxRad
Subhalo/SubhaloGasMetalFractionsSfr
Subhalo/SubhaloGasMetalFractionsSfrWeighted
Subhalo/SubhaloGasMetallicity
Subhalo/SubhaloGasMetallicityHalfRad
Subhalo/SubhaloGasMet

In [ ]:
Omega0                                            0.318
OmegaLambda                                       0.682
OmegaBaryon                                       0.048
HubbleParam                                       0.6777

In [ ]:
'/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/parameters-usedvalues'

UnitLength_in_cm                                  3.08568e+24
UnitMass_in_g                                     1.989e+43
UnitVelocity_in_cm_per_s                          100000
GasSoftFactor                                     2.8

AdaptiveHydroSofteningSpacing                     1.2
SofteningComovingType0                            0.000244
SofteningComovingType1                            0.000244
SofteningComovingType2                            0.000488
SofteningComovingType3                            0.000977
SofteningComovingType4                            0.001953
SofteningComovingType5                            0.003906
SofteningComovingType6                            0.007812
SofteningMaxPhysType0                             0.000122
SofteningMaxPhysType1                             0.000122
SofteningMaxPhysType2                             0.000244
SofteningMaxPhysType3                             0.000488
SofteningMaxPhysType4                             0.000977
SofteningMaxPhysType5                             0.001953
SofteningMaxPhysType6                             0.003906
SofteningTypeOfPartType0                          0
SofteningTypeOfPartType1                          1
SofteningTypeOfPartType2                          2
SofteningTypeOfPartType3                          3
SofteningTypeOfPartType4                          1
SofteningTypeOfPartType5                          1
SofteningTypeOfPartType6                          3

In [ ]:
h = 0.6777
G = 4.299e-6 #kpc*(km/s)*Msol^-1
a0 = 1
H0 = h*100
omega_lambda = 0.682
omega0 = 0.318
omega_matter = omega0 - omega_lambda

z = a0/aexp[isnap] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
time = tiempo.conv(z, h, omega_lambda, omega_matter)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h

v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.